In [0]:
#Milestone 7 queries

In [0]:
%run "/Workspace/Users/carla.costan0@gmail.com/mount_S3"

In [0]:
# Load the Parquet files into DataFrames
df_pin_cleaned = spark.read.parquet(f"{mount_name}/df_pin_cleaned")
df_geo_cleaned = spark.read.parquet(f"{mount_name}/df_geo_cleaned")
df_user_cleaned = spark.read.parquet(f"{mount_name}/df_user_cleaned")

# Create temporary views from the DataFrames
df_pin_cleaned.createOrReplaceTempView("df_pin_temp_view")
df_geo_cleaned.createOrReplaceTempView("df_geo_temp_view")
df_user_cleaned.createOrReplaceTempView("df_user_temp_view")


In [0]:
# Most popular category in each country

most_popular_category_by_country = spark.sql("""
   SELECT geo.country, pin.category, COUNT(*) AS category_count
   FROM df_geo_temp_view AS geo
   JOIN df_pin_temp_view AS pin 
   ON geo.ind = pin.ind
   GROUP BY geo.country, pin.category
   ORDER BY geo.country, category_count DESC
""")

display(most_popular_category_by_country)

country,category,category_count
Albania,mens-fashion,1
Armenia,diy-and-crafts,1
Aruba,tattoos,1
Azerbaijan,event-planning,1
Bulgaria,finance,1
Cocos (Keeling) Islands,vehicles,1
Colombia,finance,1
Cote d'Ivoire,education,1
Cote d'Ivoire,diy-and-crafts,1
French Guiana,quotes,1


In [0]:
# Most popular category each year

most_popular_category_by_year = spark.sql("""
    SELECT 
        YEAR(geo.timestamp) AS post_year, 
        pin.category, 
        COUNT(*) AS category_count
    FROM df_pin_temp_view AS pin
    JOIN df_geo_temp_view AS geo
    ON pin.ind = geo.ind
    WHERE YEAR(geo.timestamp) BETWEEN 2018 AND 2022
    GROUP BY post_year, pin.category
    ORDER BY post_year, category_count DESC
""")

display(most_popular_category_by_year)

post_year,category,category_count
2018,beauty,1
2019,diy-and-crafts,1
2019,education,1
2019,quotes,1
2020,diy-and-crafts,1
2020,mens-fashion,1
2020,tattoos,1
2021,finance,2
2021,event-planning,1
2022,vehicles,1


In [0]:
# User with most followers in each country

user_with_most_followers_by_country = spark.sql("""
    SELECT 
        geo.country,
        pin.poster_name AS user_name,
        MAX(pin.follower_count) AS follower_count
    FROM df_pin_temp_view AS pin
    JOIN df_geo_temp_view AS geo
    ON pin.ind = geo.ind
    GROUP BY geo.country, pin.poster_name
    ORDER BY geo.country, follower_count DESC
""")

display(user_with_most_followers_by_country)

country,user_name,follower_count
Albania,User Info Error,null
Armenia,Of Life & Lisa | Lifestyle Blog,124000.0
Aruba,TheTrendSpotter,211000.0
Azerbaijan,Style Me Pretty,6000000.0
Bulgaria,"Living Low Key | Save Money, Make Money, & Frugal Living",26000.0
Cocos (Keeling) Islands,Ray Uyemura,437.0
Colombia,Consuelo Aguirre,0.0
Cote d'Ivoire,The Crafting Chicks,192000.0
Cote d'Ivoire,Math Giraffe,25000.0
French Guiana,Commitment Connection,51000.0


In [0]:
# Most popular category for different age groups

most_popular_category_by_age_group = spark.sql("""
    WITH AgeGroups AS (
        SELECT
            user.age,
            pin.category,
            CASE 
                WHEN user.age BETWEEN 18 AND 24 THEN '18-24'
                WHEN user.age BETWEEN 25 AND 35 THEN '25-35'
                WHEN user.age BETWEEN 36 AND 50 THEN '36-50'
                WHEN user.age > 50 THEN '50+'
                ELSE 'Unknown'
            END AS age_group
        FROM df_pin_temp_view AS pin
        JOIN df_user_temp_view AS user
        ON pin.ind = user.ind
    ),
    CategoryCount AS (
        SELECT
            age_group,
            category,
            COUNT(*) AS category_count
        FROM AgeGroups
        WHERE age_group != 'Unknown'
        GROUP BY age_group, category
    )
    
    SELECT
        age_group,
        category,
        MAX(category_count) AS category_count
    FROM CategoryCount
    GROUP BY age_group, category
""")

display(most_popular_category_by_age_group)

age_group,category,category_count
18-24,tattoos,1
25-35,quotes,1
25-35,event-planning,1
36-50,education,1
25-35,finance,1
25-35,vehicles,1
25-35,diy-and-crafts,1
25-35,beauty,1
18-24,mens-fashion,1
36-50,finance,1


In [0]:
#  Median follower count by age group

median_follower_count_by_age_group = spark.sql("""
    WITH AgeGroups AS (
        SELECT
            pin.follower_count,
            CASE 
                WHEN user.age BETWEEN 18 AND 24 THEN '18-24'
                WHEN user.age BETWEEN 25 AND 35 THEN '25-35'
                WHEN user.age BETWEEN 36 AND 50 THEN '36-50'
                WHEN user.age > 50 THEN '50+'
                ELSE 'Unknown'
            END AS age_group
        FROM df_pin_temp_view AS pin
        JOIN df_user_temp_view AS user
        ON pin.ind = user.ind
        WHERE pin.follower_count IS NOT NULL AND user.age IS NOT NULL
    )
    
    SELECT
        age_group,
        PERCENTILE_APPROX(follower_count, 0.5) AS median_follower_count
    FROM AgeGroups
    GROUP BY age_group
    ORDER BY age_group
""")

display(median_follower_count_by_age_group)

age_group,median_follower_count
18-24,211000.0
25-35,43000.0
36-50,0.0


In [0]:
# Amount of new users each year

new_users_each_year = spark.sql("""
    SELECT 
        YEAR(user.date_joined) AS join_year,
        COUNT(*) AS new_user_count
    FROM df_user_temp_view AS user
    WHERE YEAR(user.date_joined) IS NOT NULL
    GROUP BY join_year
    ORDER BY join_year
""")

display(new_users_each_year)

In [0]:
# Median follower count of users based on their joining year

median_follower_count_by_join_year = spark.sql("""
    SELECT 
        YEAR(user.date_joined) AS join_year,
        PERCENTILE_APPROX(pin.follower_count, 0.5) AS median_follower_count
    FROM df_pin_temp_view AS pin
    JOIN df_user_temp_view AS user
    ON pin.ind = user.ind
    WHERE YEAR(user.date_joined) IS NOT NULL AND pin.follower_count IS NOT NULL
    GROUP BY join_year
    ORDER BY join_year
""")

display(median_follower_count_by_join_year)

In [0]:
# Median follower count of users based on their joining year and age group

median_follower_count_by_join_year_and_age_group = spark.sql("""
    WITH AgeGroups AS (
        SELECT
            YEAR(user.date_joined) AS join_year,
            pin.follower_count,
            CASE 
                WHEN user.age BETWEEN 18 AND 24 THEN '18-24'
                WHEN user.age BETWEEN 25 AND 35 THEN '25-35'
                WHEN user.age BETWEEN 36 AND 50 THEN '36-50'
                WHEN user.age > 50 THEN '50+'
                ELSE 'Unknown'
            END AS age_group
        FROM df_pin_temp_view AS pin
        JOIN df_user_temp_view AS user
        ON pin.ind = user.ind
        WHERE YEAR(user.date_joined) IS NOT NULL AND pin.follower_count IS NOT NULL AND user.age IS NOT NULL
    )
    
    SELECT
        join_year,
        age_group,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY follower_count) AS median_follower_count
    FROM AgeGroups
    GROUP BY join_year, age_group
    ORDER BY join_year, age_group
""")

display(median_follower_count_by_join_year_and_age_group)